In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Comando mágico para mostrar gráficos direto na tela
%matplotlib inline 
sns.set_theme(style="whitegrid") # Estilo visual limpo

# --- CAMINHOS ---
INPUT_FILE = "/home/marcos/PRJEB59406/tabela_mestra_mag_sialidase.xlsx"
OUTPUT_DIR = "/home/marcos/PRJEB59406/mag_annotation/plots"

os.makedirs(OUTPUT_DIR, exist_ok=True)
print("Ambiente configurado.")

In [ ]:
print("Carregando e processando tabela...")

try:
    df = pd.read_excel(INPUT_FILE)
    
    # --- FUNÇÃO DE LIMPEZA ---
    def parse_taxonomy(row):
        """Quebra a string 'd__Bacteria;p__Firmicutes...' em colunas"""
        if pd.isna(row): return pd.Series(['Unknown', 'Unknown'])
        
        parts = row.split(';')
        tax_dict = {}
        for part in parts:
            if '__' in part:
                rank, name = part.split('__', 1) # Separa no primeiro '__'
                tax_dict[rank] = name
        
        # Retorna Filo (p) e Gênero (g) - Adicione outros se quiser
        return pd.Series([tax_dict.get('p', 'Unknown'), tax_dict.get('g', 'Unknown')])

    # Aplica a função
    df[['Phylum', 'Genus']] = df['classification'].apply(parse_taxonomy)
    
    # Remove prefixos vazios (às vezes o GTDB deixa 'p__')
    df['Phylum'] = df['Phylum'].replace('', 'Unknown')
    
    print(f"Total de MAGs carregados: {len(df)}")
    
    # MOSTRA A TABELA PARA CONFERÊNCIA
    display(df[['mag_id', 'Phylum', 'Genus', 'has_sialidase']].head())

except Exception as e:
    print(f"Erro: {e}")

In [ ]:
plt.figure(figsize=(10, 8))

# Conta MAGs por Filo e ordena do maior para o menor
order = df['Phylum'].value_counts().index

sns.countplot(data=df, y='Phylum', order=order, palette='viridis')

plt.title("Diversidade dos MAGs Montados (Nível de Filo)", fontsize=16)
plt.xlabel("Quantidade de Genomas (MAGs)")
plt.ylabel("Filo")

# Salva e Mostra
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/mag_distribuicao_filos.png", dpi=300)
plt.show()

In [ ]:
# Filtra apenas os top 12 Filos para o gráfico não ficar ilegível
top_phyla = df['Phylum'].value_counts().head(12).index
df_filtered = df[df['Phylum'].isin(top_phyla)]

plt.figure(figsize=(12, 8))

# Gráfico comparativo
sns.countplot(data=df_filtered, y='Phylum', hue='has_sialidase', 
              order=top_phyla, 
              palette={'YES': '#e74c3c', 'NO': '#95a5a6', 'No Data': '#ecf0f1'}) 
              # Vermelho para YES, Cinza para NO

plt.title("Presença de Sialidase nos Principais Filos Bacterianos", fontsize=16)
plt.xlabel("Quantidade de MAGs")
plt.ylabel("Filo")
plt.legend(title="Possui Sialidase?", loc='lower right')

# Salva e Mostra
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/mag_sialidase_presence.png", dpi=300)
plt.show()

# Extra: Mostra quem são os positivos
print("Lista de Gêneros que possuem Sialidase:")
sialidase_positives = df[df['has_sialidase'] == 'YES']['Genus'].unique()
print(sialidase_positives)